In [6]:
!pip install numpy


In [23]:
from pyspark.sql import SparkSession
from pathlib import Path
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, hour, dayofweek, dayofmonth, avg, count, sum as spark_sum, desc, asc
from pyspark.sql.functions import round as spark_round, when, isnan, isnull
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
import pandas as pd

# Create SparkSession in local mode
spark = SparkSession.builder \
    .appName("FareTipModeling") \
    .master("local[*]") \
    .getOrCreate()

In [11]:
folder_path = Path(r"\Your-path\data\cleanedData")
parquet_files = sorted(folder_path.glob("*.parquet"))

print(f"Found {len(parquet_files)} parquet files:")
all_dfs = []
for f in parquet_files:
    print(f"\nReading: {f.name}")
    df = spark.read.parquet(str(f))
    all_dfs.append(df)
    df.show(5)

Found 9 parquet files:

Reading: part-00000-fde1b9ee-d448-4a3f-bbd6-7aff4e42f554-c000.snappy.parquet
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2024-08-01 00:21:00|  2024-08-01 00:36:13|              1|          7.4|   

In [12]:
# Combine all DataFrames
if len(all_dfs) > 1:
    combined_df = all_dfs[0]
    for df in all_dfs[1:]:
        combined_df = combined_df.union(df)
else:
    combined_df = all_dfs[0]

print(f"\nCombined dataset shape: {combined_df.count()} rows, {len(combined_df.columns)} columns")
print("\nCombined dataset schema:")
combined_df.printSchema()


Combined dataset shape: 32036273 rows, 19 columns

Combined dataset schema:
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [37]:
# MEMBER 3: GEOSPATIAL AND ROUTE ANALYSIS
# ========================================

print("\n" + "="*60)
print("MEMBER 3: GEOSPATIAL AND ROUTE ANALYSIS")
print("="*60)

# Step 1: Data Quality Check for Location IDs
print("\n1. DATA QUALITY CHECK FOR LOCATION IDs")
print("-" * 40)
# Check for null/invalid location IDs
null_pickup = combined_df.filter(col("PULocationID").isNull() | (col("PULocationID") == 0)).count()
null_dropoff = combined_df.filter(col("DOLocationID").isNull() | (col("DOLocationID") == 0)).count()

print(f"Records with null/zero pickup location: {null_pickup}")
print(f"Records with null/zero dropoff location: {null_dropoff}")


MEMBER 3: GEOSPATIAL AND ROUTE ANALYSIS

1. DATA QUALITY CHECK FOR LOCATION IDs
----------------------------------------
Records with null/zero pickup location: 0
Records with null/zero dropoff location: 0


In [15]:
# Filter out invalid location records
valid_trips = combined_df.filter(
    col("PULocationID").isNotNull() & 
    col("DOLocationID").isNotNull() & 
    (col("PULocationID") > 0) & 
    (col("DOLocationID") > 0)
)

print(f"Valid trips after filtering: {valid_trips.count()}")

Valid trips after filtering: 32036273


In [38]:
# Step 2: Zone Analysis - Most Popular Pickup and Dropoff Zones
print("\n2. ZONE ANALYSIS - PICKUP AND DROPOFF PATTERNS")
print("-" * 50)
# Most popular pickup zones
print("Top 15 Pickup Zones:")
pickup_zones = valid_trips.groupBy("PULocationID") \
    .agg(count("*").alias("pickup_count")) \
    .orderBy(desc("pickup_count"))

pickup_zones.show(15)


2. ZONE ANALYSIS - PICKUP AND DROPOFF PATTERNS
--------------------------------------------------
Top 15 Pickup Zones:
+------------+------------+
|PULocationID|pickup_count|
+------------+------------+
|         132|     1803888|
|         161|     1541646|
|         237|     1502832|
|         236|     1332390|
|         138|     1224279|
|         186|     1169233|
|         162|     1164590|
|         230|     1122802|
|         142|     1067187|
|         163|      950142|
|         170|      929948|
|          68|      897902|
|         239|      878162|
|         234|      873885|
|          48|      814407|
+------------+------------+
only showing top 15 rows


In [18]:
# Most popular dropoff zones
print("\nTop 15 Dropoff Zones:")
dropoff_zones = valid_trips.groupBy("DOLocationID") \
    .agg(count("*").alias("dropoff_count")) \
    .orderBy(desc("dropoff_count"))

dropoff_zones.show(15)


Top 15 Dropoff Zones:
+------------+-------------+
|DOLocationID|dropoff_count|
+------------+-------------+
|         236|      1406717|
|         237|      1324167|
|         161|      1179946|
|         230|       998733|
|         170|       936607|
|         142|       915052|
|         162|       914785|
|         239|       907308|
|          68|       839403|
|         141|       825656|
|         163|       799240|
|         234|       766654|
|          48|       763469|
|         238|       719371|
|         186|       699714|
+------------+-------------+
only showing top 15 rows


In [39]:
# Step 3: Route Analysis - Most Common Routes
print("\n3. ROUTE ANALYSIS - MOST COMMON ROUTES")
print("-" * 45)
# First, let's check what columns are available
print("Available columns in the dataset:")
print(combined_df.columns)

# Calculate trip duration from pickup and dropoff times
from pyspark.sql.functions import unix_timestamp, to_timestamp

# Add trip duration calculation
trips_with_duration = valid_trips.withColumn(
    "trip_duration_minutes",
    (unix_timestamp("tpep_dropoff_datetime") - unix_timestamp("tpep_pickup_datetime")) / 60
)


3. ROUTE ANALYSIS - MOST COMMON ROUTES
---------------------------------------------
Available columns in the dataset:
['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'total_amount', 'congestion_surcharge', 'Airport_fee']


In [26]:
 # Analyze most common pickup-dropoff zone pairs
routes_analysis = trips_with_duration.groupBy("PULocationID", "DOLocationID") \
    .agg(
        count("*").alias("trip_count"),
        avg("trip_distance").alias("avg_distance"),
        avg("trip_duration_minutes").alias("avg_duration_minutes"),
        avg("fare_amount").alias("avg_fare")
    ) \
    .filter(col("trip_count") >= 10) \
    .orderBy(desc("trip_count"))

print("Top 20 Most Common Routes (Pickup -> Dropoff):")
routes_analysis.select(
    col("PULocationID").alias("Pickup_Zone"),
    col("DOLocationID").alias("Dropoff_Zone"),
    col("trip_count"),
    spark_round("avg_distance", 2).alias("avg_distance_miles"),
    spark_round("avg_duration_minutes", 2).alias("avg_duration_min"),
    spark_round("avg_fare", 2).alias("avg_fare_$")
).show(20)

Top 20 Most Common Routes (Pickup -> Dropoff):
+-----------+------------+----------+------------------+----------------+----------+
|Pickup_Zone|Dropoff_Zone|trip_count|avg_distance_miles|avg_duration_min|avg_fare_$|
+-----------+------------+----------+------------------+----------------+----------+
|        237|         236|    227393|              1.14|            8.21|      9.29|
|        236|         237|    192119|              1.12|            9.19|      9.81|
|        161|         237|    107610|              1.14|           10.74|     10.76|
|        161|         236|    102031|              1.93|           14.28|      14.3|
|        237|         161|     92715|              1.12|            11.6|     11.28|
|        132|         230|     89236|             17.98|           61.37|     70.65|
|        142|         239|     82732|              1.07|            7.24|       8.5|
|        237|         162|     81886|              1.08|           10.25|     10.39|
|        236|     

In [28]:
 # Step 4: Trip Duration Analysis by Zone Pairs
print("\n4. TRIP DURATION ANALYSIS BY ZONE PAIRS")
print("-" * 45)

# Find routes with longest average duration
longest_duration_routes = routes_analysis \
    .filter(col("trip_count") >= 50) \
    .orderBy(desc("avg_duration_minutes"))

print("Routes with Longest Average Duration (min 50 trips):")
longest_duration_routes.select(
    col("PULocationID").alias("Pickup_Zone"),
    col("DOLocationID").alias("Dropoff_Zone"),
    col("trip_count"),
    spark_round("avg_duration_minutes", 2).alias("avg_duration_min"),
    spark_round("avg_distance", 2).alias("avg_distance_miles")
).show(15)



4. TRIP DURATION ANALYSIS BY ZONE PAIRS
---------------------------------------------
Routes with Longest Average Duration (min 50 trips):
+-----------+------------+----------+----------------+------------------+
|Pickup_Zone|Dropoff_Zone|trip_count|avg_duration_min|avg_distance_miles|
+-----------+------------+----------+----------------+------------------+
|         68|          86|        77|          115.34|             20.98|
|        107|          91|        58|          109.53|             10.58|
|         55|          41|        56|          108.33|             19.91|
|        137|         203|       126|          102.24|             17.15|
|         61|          69|        51|          100.66|             14.21|
|        232|         226|        55|          100.01|              5.33|
|        117|         209|        59|           99.42|             17.48|
|         88|          51|        64|            99.0|              16.8|
|        162|          76|       103|         

In [29]:
# Find routes with shortest average duration
shortest_duration_routes = routes_analysis \
    .filter(col("trip_count") >= 50) \
    .orderBy(asc("avg_duration_minutes"))

print("\nRoutes with Shortest Average Duration (min 50 trips):")
shortest_duration_routes.select(
    col("PULocationID").alias("Pickup_Zone"),
    col("DOLocationID").alias("Dropoff_Zone"),
    col("trip_count"),
    spark_round("avg_duration_minutes", 2).alias("avg_duration_min"),
    spark_round("avg_distance", 2).alias("avg_distance_miles")
).show(15)


Routes with Shortest Average Duration (min 50 trips):
+-----------+------------+----------+----------------+------------------+
|Pickup_Zone|Dropoff_Zone|trip_count|avg_duration_min|avg_distance_miles|
+-----------+------------+----------+----------------+------------------+
|        207|         138|        68|            3.81|              2.06|
|        129|         138|      1184|            4.92|              2.23|
|        166|          24|      2097|            5.74|              0.95|
|        151|          24|      3648|            5.79|               0.9|
|        152|         166|      1732|            5.88|              0.99|
|        238|         151|     23286|            5.96|               0.9|
|        262|         263|     12354|            6.03|              0.87|
|         24|         151|      1717|            6.06|              0.89|
|        238|          24|     12256|            6.19|              1.03|
|        224|         137|      1479|            6.22|   

In [30]:
# Step 5: Zone Activity Analysis
print("\n5. ZONE ACTIVITY ANALYSIS")
print("-" * 30)

# Combine pickup and dropoff activity for each zone
pickup_activity = pickup_zones.select(col("PULocationID").alias("LocationID"), col("pickup_count"))
dropoff_activity = dropoff_zones.select(col("DOLocationID").alias("LocationID"), col("dropoff_count"))

zone_activity = pickup_activity.join(dropoff_activity, "LocationID", "outer") \
    .fillna(0) \
    .select(
        col("LocationID"),
        col("pickup_count"),
        col("dropoff_count"),
        (col("pickup_count") + col("dropoff_count")).alias("total_activity")
    ) \
    .orderBy(desc("total_activity"))

print("Top 15 Most Active Zones (Pickup + Dropoff combined):")
zone_activity.show(15)


5. ZONE ACTIVITY ANALYSIS
------------------------------
Top 15 Most Active Zones (Pickup + Dropoff combined):
+----------+------------+-------------+--------------+
|LocationID|pickup_count|dropoff_count|total_activity|
+----------+------------+-------------+--------------+
|       237|     1502832|      1324167|       2826999|
|       236|     1332390|      1406717|       2739107|
|       161|     1541646|      1179946|       2721592|
|       132|     1803888|       350769|       2154657|
|       230|     1122802|       998733|       2121535|
|       162|     1164590|       914785|       2079375|
|       142|     1067187|       915052|       1982239|
|       186|     1169233|       699714|       1868947|
|       170|      929948|       936607|       1866555|
|       239|      878162|       907308|       1785470|
|       163|      950142|       799240|       1749382|
|        68|      897902|       839403|       1737305|
|       234|      873885|       766654|       1640539|
|       

In [31]:
# Step 6: Distance vs Duration Analysis by Routes
print("\n6. DISTANCE VS DURATION EFFICIENCY ANALYSIS")
print("-" * 50)

# Calculate speed (distance/duration) for different routes
route_efficiency = routes_analysis \
    .filter((col("avg_distance") > 0) & (col("avg_duration_minutes") > 0)) \
    .select(
        col("PULocationID").alias("Pickup_Zone"),
        col("DOLocationID").alias("Dropoff_Zone"),
        col("trip_count"),
        spark_round("avg_distance", 2).alias("avg_distance_miles"),
        spark_round("avg_duration_minutes", 2).alias("avg_duration_min"),
        spark_round((col("avg_distance") / (col("avg_duration_minutes") / 60)), 2).alias("avg_speed_mph")
    ) \
    .filter(col("trip_count") >= 30)

print("Routes with Highest Average Speed (min 30 trips):")
route_efficiency.orderBy(desc("avg_speed_mph")).show(15)

print("\nRoutes with Lowest Average Speed (min 30 trips):")
route_efficiency.orderBy(asc("avg_speed_mph")).show(15)


6. DISTANCE VS DURATION EFFICIENCY ANALYSIS
--------------------------------------------------
Routes with Highest Average Speed (min 30 trips):
+-----------+------------+----------+------------------+----------------+-------------+
|Pickup_Zone|Dropoff_Zone|trip_count|avg_distance_miles|avg_duration_min|avg_speed_mph|
+-----------+------------+----------+------------------+----------------+-------------+
|        223|         134|        54|              7.07|           10.61|        39.96|
|        265|           1|       585|              9.81|           15.64|        37.63|
|        130|         223|        46|              8.98|           14.87|        36.26|
|          1|         265|        60|              13.9|           23.19|        35.97|
|        132|          44|        65|             37.45|           62.85|        35.76|
|        130|         260|        41|              7.95|           13.44|        35.52|
|        223|          93|        44|               5.7|      

In [32]:
# Step 7: Self-Zone Trips Analysis
print("\n7. SELF-ZONE TRIPS ANALYSIS")
print("-" * 35)

# Analyze trips within the same zone
same_zone_trips = trips_with_duration.filter(col("PULocationID") == col("DOLocationID")) \
    .groupBy("PULocationID") \
    .agg(
        count("*").alias("same_zone_trips"),
        avg("trip_distance").alias("avg_distance"),
        avg("trip_duration_minutes").alias("avg_duration"),
        avg("fare_amount").alias("avg_fare")
    ) \
    .orderBy(desc("same_zone_trips"))

print("Zones with Most Same-Zone Trips:")
same_zone_trips.select(
    col("PULocationID").alias("Zone_ID"),
    col("same_zone_trips"),
    spark_round("avg_distance", 2).alias("avg_distance_miles"),
    spark_round("avg_duration", 2).alias("avg_duration_min"),
    spark_round("avg_fare", 2).alias("avg_fare_$")
).show(15)


7. SELF-ZONE TRIPS ANALYSIS
-----------------------------------
Zones with Most Same-Zone Trips:
+-------+---------------+------------------+----------------+----------+
|Zone_ID|same_zone_trips|avg_distance_miles|avg_duration_min|avg_fare_$|
+-------+---------------+------------------+----------------+----------+
|    237|          78358|               0.9|            8.24|      8.74|
|    236|          65326|              0.94|            7.27|      8.28|
|    264|          51539|              3.51|           20.05|     20.02|
|     68|          29185|              1.05|            8.17|      8.92|
|    132|          25815|              4.75|           21.27|     27.53|
|    141|          23533|              0.95|            7.47|      8.45|
|    239|          20955|              1.02|            7.36|      8.67|
|     48|          20791|              1.01|            8.84|      9.33|
|     43|          20592|               1.3|           10.33|     10.49|
|    161|          20404| 

In [36]:
# Step 8: Prepare Aggregated Route Data for Visualization
print("\n8. PREPARING AGGREGATED ROUTE DATA FOR VISUALIZATION")
print("-" * 55)

# Create summary statistics for visualization team (Member 5)
route_summary = routes_analysis.select(
    col("PULocationID"),
    col("DOLocationID"),
    col("trip_count"),
    spark_round("avg_distance", 3).alias("avg_distance"),
    spark_round("avg_duration_minutes", 3).alias("avg_duration"),
    spark_round("avg_fare", 3).alias("avg_fare")
).filter(col("trip_count") >= 20)  # Filter for significant routes

print(f"Route summary prepared with {route_summary.count()} significant routes")
print("Sample of route summary data:")
route_summary.show(10)





8. PREPARING AGGREGATED ROUTE DATA FOR VISUALIZATION
-------------------------------------------------------
Route summary prepared with 16540 significant routes
Sample of route summary data:
+------------+------------+----------+------------+------------+--------+
|PULocationID|DOLocationID|trip_count|avg_distance|avg_duration|avg_fare|
+------------+------------+----------+------------+------------+--------+
|         237|         236|    227393|       1.135|       8.214|   9.292|
|         236|         237|    192119|       1.116|       9.188|   9.814|
|         161|         237|    107610|       1.141|      10.738|  10.756|
|         161|         236|    102031|       1.927|      14.282|  14.303|
|         237|         161|     92715|       1.125|      11.597|  11.282|
|         132|         230|     89236|      17.977|      61.367|  70.652|
|         142|         239|     82732|       1.075|        7.24|   8.503|
|         237|         162|     81886|       1.081|      10.253|  1